# Running the gerrychain to find a map without holes on congressional districts

@authors: vcle, bpuhani

In [ ]:
import io
from contextlib import redirect_stdout

import maup
import pandas as pd

import utilities as util
import matplotlib.pyplot as plt
import geopandas as gpd
import numpy as np
from gerrychain import Graph, Partition, proposals, updaters, constraints, accept, MarkovChain, Election
from gerrychain.tree import bipartition_tree
from gerrychain.updaters import cut_edges, Tally
from gerrychain.proposals import recom, propose_random_flip
from gerrychain.accept import always_accept
from functools import partial

## Loading the needed data.
For this notebook to work we assume, that you ran the following notebooks first:
* `0_IL_import_and_explore_data.ipynb`
* `B_2_IL_clean_maup_with_congress.ipynb`

In [ ]:
il_df: gpd.GeoDataFrame = util.load_shapefile("il_data/IL_congress_without_holes.shp")
il_graph: Graph = util.load_graph("il_data/IL_congress_without_holes.shp")

In [ ]:
# Set up the initial partition object
initial_partition = Partition(
    il_graph,
    assignment="CONGD",  # as per assignment
    updaters={
        # setup updaters, that get updated per run of the chain
        "total_population": Tally("TOTPOP", alias="total_population"),
        # TODO set updater for hispanic population
        "cut_edges": cut_edges
        # TODO add elections for PRE and USS
    }
)

In [ ]:
# Define the ideal population
ideal_population = sum(initial_partition["total_population"].values()) / len(initial_partition)
print("Nr of districts:", len(initial_partition))
print("Ideal population:", ideal_population)

In [ ]:
# Define the recom proposal
proposal = partial(
    recom,
    pop_col="TOTPOP",
    pop_target=ideal_population,
    epsilon=0.02,
    method=partial(
        bipartition_tree,
        max_attempts=100,
        allow_pair_reselection=True
    )
)

In [ ]:
def run_the_chain(nr_of_total_steps: int, start_partition: Partition, offset: int = 0) -> Partition:
    """Runs the chain for the specified number of steps. Returns the last partition"""

    # Set up the chain
    chain = MarkovChain(
        proposal=proposal,
        constraints=[
            # Compactness constraint
            constraints.UpperBound(lambda p: len(p["cut_edges"]), 2 * len(initial_partition["cut_edges"])),
            # Population constraint
            constraints.within_percent_of_ideal_population(initial_partition, 0.02, "total_population")
            # TODO set constraint for the map not to allow holes
        ],
        accept=always_accept,
        initial_state=start_partition,
        total_steps=nr_of_total_steps
    )
    last_partition: Partition = start_partition

    for (i, partition) in enumerate(chain.with_progress_bar()):
        last_partition = partition

        #print(f"There are {partition.nr_of_holes} holes overall")
        # append the number of cut edges for this proposal to the list



    return last_partition

In [ ]:
partition_at_10_000 = run_the_chain(10_000, initial_partition)

Verify the result:
(This result is not reproducible)

In [ ]:
districts_df: gpd.GeoDataFrame = gpd.GeoDataFrame()

# create a districts dataFrame from the partition
for district, nodes in partition_at_10_000.parts.items():
        records = [partition_at_10_000.graph.nodes[n] for n in nodes]
        this_district_df = gpd.GeoDataFrame(records)
        this_district_df["district"] = district
        districts_df = gpd.GeoDataFrame(pd.concat([districts_df, this_district_df], ignore_index=True))

In [ ]:
# set the crf to the same as the original dataframe
districts_df.crs = il_df.crs

In [ ]:
print(len(districts_df))
print(len(il_df))

In [ ]:
print(f"{sum(il_df['TOTPOP']):_}")  # check if there are any holes

In [ ]:
print(f"{sum(districts_df['TOTPOP']):_}")  # check if there are any holes

In [ ]:
print(set(districts_df["district"]))

In [ ]:
# Plot map
title = "Illinois Congressional map without holes"
districts_df.plot(column="district", cmap="tab20", figsize=(12, 12), legend=False)

# Final touches
plt.axis("off")
plt.title(title, fontsize=20)
plt.tight_layout()
plt.savefig(f"images/illinois_map_{title}.svg", bbox_inches="tight")
plt.show()
plt.close()